# Rational Strain Design

Author: Daniel Machado [(CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/)

- In this tutorial you will learn how to use metabolic models and flux balance analysis for rational strain design.
- You will use the [ReFramed](https://github.com/cdanielmachado/reframed) and [MEWpy](https://github.com/BioSystemsUM/mewpy) python libraries. Please check their respective documentation for details.
- If you haven't done so, you should try the [FBA tutorial](1_fba_intro.ipynb) first :)

## Step 1: Loading a model stored in SBML format

Again, we will use the central carbon metabolism model of *E. coli*.

In [ ]:
from reframed import load_cbmodel
model = load_cbmodel('../files/e_coli_core.xml')

## Step 2: Defining the problem

Imagine you want to overproduce **succinate** under **anaerobic** conditions. 

Let's begin by completely removing oxygen from the growth medium and check how much succinate is overproduced (i.e. secreted) by the wild-type strain:

In [ ]:
from reframed import FBA
model.reactions.R_EX_o2_e.lb = 0
solution = FBA(model)
print(f"Growth rate: {solution.fobj}")
print(f"Succinate secretion: {solution.values['R_EX_succ_e']}")

As we can see, the wild-type strain is not secreting any succinate.

We can also query the model with regard to the maximum theoretical production instead. We can do this by setting succinate secretion as our objective function in FBA:

In [ ]:
solution = FBA(model, objective='R_EX_succ_e')
print(f"Succinate secretion: {solution.values['R_EX_succ_e']}")

To better understand the trade-off between growth and production we can look at the **production envelope**: 

In [ ]:
from reframed import plot_flux_envelope
plot_flux_envelope(model, model.biomass_reaction, 'R_EX_succ_e')

We can see there is a trade-off between growing and secreting succinate, evolution is not on our side.

## Step 3: Simulating multiple gene/reaction deletions

We can try to re-direct metabolic fluxes using gene (or reaction) deletions.

Let's start with a triple gene knockout:

In [ ]:
from reframed import gene_knockout
solution = gene_knockout(model, ['G_b0008', 'G_b2935', 'G_b4090'])

print(f"Growth rate: {solution.fobj}")
print(f"Succinate secretion: {solution.values['R_EX_succ_e']}")

Well, that didn't seem to help. 

Let's try a quadruple reaction knockout this time.

In [ ]:
from reframed import reaction_knockout
solution = reaction_knockout(model, ['R_PFL', 'R_LDH_D', 'R_ACALD', 'R_ACKr'])

print(f"Growth rate: {solution.fobj}")
print(f"Succinate secretion: {solution.values['R_EX_succ_e']}")

Ok, that worked. But why? 

Let's look at the production envelope again:

In [ ]:
knockouts = {'R_PFL':0, 'R_LDH_D':0, 'R_ACALD':0, 'R_ACKr':0}
plot_flux_envelope(model, model.biomass_reaction, 'R_EX_succ_e', constraints=knockouts)

This is what we call a growth-coupled design. This *E. coli* mutant **must** secrete succinate in order to grow.

Let's see how this looks in terms of flux distribution.

In [ ]:
from reframed import fluxes2escher
solution = FBA(model, constraints=knockouts)
fluxes2escher(solution.values)

## Step 4: Finding optimal sets of gene/reaction deletions using MEWpy

Ok, but how did we know that particular combination of deletions would work? 

We can simply try to simulate all possible combinations of gene/reaction deletions, but this can explode very quickly. For instance, the latest E. coli model contains 1515 metabolic genes. That's a lot of simulations:

- double deletions: $1.14 *10^6$
- triple deletions: $5.78 *10^8$
- quadruple deletions: $2.19 *10^{11}$
- *etc*...

When we encounter this kind of combinatorial optimization problem, we can apply [metaheuristics](https://en.wikipedia.org/wiki/Metaheuristic). This is a particular type of algorithms to solve optimization problems that try to find a **good enough** solution in a reasonable amount of time.

![metaheuristics](../files/metaheuristics.png)
Image source: wikipedia

-------

### MEWpy basics

**MEWpy** is a strain design library for metabolic models that implements a few different metaheuristic optimization methods (take a look at the [documentation](https://mewpy.readthedocs.io/en/latest/main.html)).

It supports the evaluation of multiple design objectives. We will begin by defining two objectives:

* Maximize the flux of our target reaction (succinate production)
* Maximize the Biomass-Product Coupled Yield (finds an optimal trade-off between growth and production)

In [ ]:
from mewpy.optimization.evaluation import BPCY, TargetFlux

model = load_cbmodel('../files/e_coli_core.xml')

objectives = [
    TargetFlux("R_EX_succ_e"), 
    BPCY(model.biomass_reaction, "R_EX_succ_e")
]

Now we define our optimization problem:
* Type of modifications we are searching for (we will use gene knockouts)
* Environmental conditions (anaerobic growth)

In [ ]:
from mewpy.problems import GKOProblem
anaerobic = {'R_EX_o2_e': (0, 0)} 

problem = GKOProblem(model, fevaluation=objectives, envcond=anaerobic)

# for reaction knockouts use RKOProblem instead
#
# from mewpy.problems import RKOProblem
# problem = RKOProblem(model, fevaluation=objectives, envcond=anaerobic)

And now we can run the optimization as an [evolutionary algorithm (EA)](https://en.wikipedia.org/wiki/Evolutionary_algorithm):

> **Note**: These algorithms are stochastic, so if you don't get good results in the first attempt, try running a few more times or increase the number of generations.

In [ ]:
from mewpy.optimization import EA
solutions = EA(problem).run()

# solutions = EA(problem, 
#                max_generations=100, # we can increase the number of generations for the EA
#                population_size=100  # and we can also try to increase the population size
#               ).run() # the computation time will increase proportionally (population * generations) 

By default, MEWpy calculates a population of the best 100 solutions.

To make our life easier, let's convert the result to a Pandas DataFrame:

In [ ]:
import pandas as pd

get_list = lambda x: [r_id[2:] for r_id in x.values]
table = [[get_list(x), len(get_list(x)), x.fitness[0], x.fitness[1]] for x in solutions]
df = pd.DataFrame(table, columns=["knockouts", "total", "rate", "BPCY"])

We can now sort the results, for instance by the total number of required knockouts:

In [ ]:
df.sort_values("total")

We can also look at the trade-off between our two objectives (the so-called [Pareto front](https://en.wikipedia.org/wiki/Pareto_front)):

In [ ]:
df.plot.scatter("rate", "BPCY")

> Which of these solutions would **you** implement in the lab ?